# Social Network Analysis of Swiss Politicians on Twitter Data
## Tasks
In this assignment you will do the following tasks:
1. Construct the timelines of Twitter users
2. Build social network of retweets
3. Calculate assortativity
4. Permutation tests
5. Community detection

### Install requirements. 

The following cell contains all the necessary dependencies needed for this task. If you run the cell everything will be installed.  

* [`twarc`](https://twarc-project.readthedocs.io/en/latest/) is a Python package for collecting and archiving Twitter JSON data via the Twitter API. [Here](https://twarc-project.readthedocs.io/en/latest/api/client2/) is the documentation of `twarc`.
* [`pandas`](https://pandas.pydata.org/docs/index.html) is a Python package for creating and working with tabular data. [Here](https://pandas.pydata.org/docs/reference/index.html) is the documentation of `pandas`.
* [`numpy`](https://numpy.org/) is a Python package for mathematical functions. [Here](https://numpy.org/doc/stable/reference/index.html) is the documentation of `numpy`.
* [`matplotlib`](https://matplotlib.org/) is a Python package for creating plots. [Here](https://matplotlib.org/stable/api/index.html) is the documentation of `matplotlib`.
* [`networkx`](https://networkx.org/) is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks. [Here](https://networkx.org/documentation/stable/reference/index.html) is the documentation of `networkx`.

In [ ]:
! pip install twarc
! pip install pandas
! pip install numpy
! pip install matplotlib
! pip install networkx

### Import requirements
The cell below imports all necessary dependancies. Make sure they are installed (see cell above).

In [ ]:
import pandas as pd
from twarc import Twarc2
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone

# 1. Construct the timelines of Twitter users

First connect to the Twitter API using your credentials. See [here](https://scholarslab.github.io/learn-twarc/02-twitter-setup.html#accessing-keys-and-tokens) how to get your bearer token (You can generate one in the Key and Tokens section of your app).

In [ ]:
bearer_token = "XXX" # replace the XXX with your bearer token
twarc_client = Twarc2(bearer_token=bearer_token)

Import the file SwissPoliticians.csv and read it as a csv. Take into account that separators are tabs.

In [ ]:
# Your Code goes here!


Look up the basic profile information of each user by screenname (see function `user_lookup()` in [twarc2](https://twarc-project.readthedocs.io/en/latest/api/client2/#twarc.client2.Twarc2.user_lookup)).

In [ ]:
# Your Code goes here!


Merge the user information you retrieved from Twitter with the party affiliations from the `SwissPoliticians.csv` file. Remove all protected users from the dataset and save the user dataset to disk (users where the `"protected"` column is set to True). For the merging you can use `pandas` [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) method.

Hint: convert the screen names in both data frames to lower case, to prevent any merging issues. Here you can use `pandas` [`str.lower`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.lower.html) method.

You can also use the provided `users.csv` file if you not manage to download and edit the data.

In [ ]:
# Your Code goes here!


Now retrieve the all tweets from the last 5 days for each user using [twarc](https://twarc-project.readthedocs.io/en/latest/api/client2/#twarc.client2.Twarc2.timeline)'s `timeline()` function. Since we want to build a retweet network, we need to know the user ID of the original tweet for every retweet. To get this information, we need to request an *[expansion](https://developer.twitter.com/en/docs/twitter-api/expansions)*.

It might take a bit to get data. If you run unto the [rate limit](https://developer.twitter.com/en/docs/twitter-api/rate-limits) of the Twitter API, you might have to wait up to 15 min to retrieve all tweets. Twarc will wait and resume the request automatically and print a warning.

Save the result in a file called `timelines.csv` so you can reload it at a later point in time.

**Note:** if you do not manage to download the tweets, we also provide a `timelines.csv` file for you that includes tweets from July 5 to July 12. You can proceed to the next step to load this dataset and continue working with it.

In [ ]:
# Your Code goes here!


since we requested an expansion in the `referenced_tweets.id` field, we have a rather complicated nested JSON structure now. Use the [`ensure_flattened`](https://twarc-project.readthedocs.io/en/latest/api/expansions/) utility from `twarc` to flatten the JSON structure into a more manageable format.

In [ ]:
# Your Code goes here!


Parse the JSON into a .csv and save it. Use `pandas` [`json_normalize`](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html) to make absolutely sure the data is flatten.

In [ ]:
# Your Code goes here!


# 2. Build social networks of retweets

First load the timelines and users from the files, parse the creation date as datetime and convert the IDs to srings to avoid int overflows. Use `pandas` [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) to do so. Especcialy look at the keyword arguments `dtype` and `parse_dates`.

 Since the `referenced_tweets` field contains a list of dictionaries that is stored as a string, we need to parse it first to restore its structure as list of dictionaries to interact with it. For this you can use a combination of pythons [`eval`](https://docs.python.org/3/library/functions.html#eval) function and `pandas` [`apply`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method. This function takes a string and evaluates it as a python expression. For example if you have following String:
```
some_string_with_a_dict = '{"first_key": "first_value",
                            "second_key": "second_value",
                            "third_key": "third_value",}'
```
And use `eval` on it like the following:
```
usable_dictionary_from_string = eval(some_string_with_a_dict)
```
You get the dictionary as a python local usable dictonary. Try it out!
Now you can convert every `"referenced_tweets"` field in the dataset to usable python expressions.

In [ ]:
# Your Code goes here!


The field `referenced_tweets` currently contains a list where each entry is a tweet object (since we requested the expansion on the field `referenced_tweets.id`.

To construct our retweet network, we need to know (a) whether a tweet was a retweet and (b) the ID of the account that posted the tweet that was retweeted. Below we define two functions that help us extract this information from the `referenced_tweets` field:

In [ ]:
def check_retweet(entry):
    '''Checks whether a tweet is a retweet'''
    if entry != entry: # NaN check
        return False
    for reference in entry:
        if reference["type"] == "retweeted":
            return True
    return False

def get_retweet_author(entry):
    '''Returns the author ID of the retweeted tweet'''
    if entry != entry: # NaN check
        return np.nan
    for reference in entry:
        if reference["type"] == "retweeted":
            return reference["author_id"]
    return np.nan

Apply the functions `check_retweet()` and `get_retweet_author` to the column `referenced_tweets` and create two new columns `retweeted` and `retweet_user_id` containing the relevant information. Again you can use `pandas` [`apply`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method.

In [ ]:
# Your Code goes here!


Filter the tweets in the timelines such that you only retain retweets.

In [ ]:
# Your Code goes here!


Now filter the timelines such that the `retweet_user_id` is one of the user IDs in the user list. Use `pandas` [`isin`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html) method for this.

In [ ]:
# Your Code goes here!


Now finaly we can start to create the Graph, start by creating an empty graph with `networkx`. See [here](https://networkx.org/documentation/stable/reference/introduction.html#graph-creation) for more information.

In [ ]:
# Your Code goes here!


To build the retweet network, we have to fill the empty graph object we just created with nodes and edges. For this purpose, we prepare a list of nodes and their attributes and a list of edges.

First, construct a list of vertices (nodes) and node attributes containing the user ids,  screen_names, and the **political party label** of the vertices. Remove all users without a party. Each entry of the list has the following form: 

`(id, {"username":username, "party":party})`

Use the function [`add_nodes_from`](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.add_nodes_from.html) provided by `networkx` to add the nodes to the graph. 

Then build a list of edges where every edge is a pair of two users that exchanged at least one retweet with each other (regardless of the direction, remove all duplicate entries). Each entry of the list has the following form:  

`(author_id_1, author_id_2)`

Use the function [`add_edges_from`](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.add_edges_from.html) provided by `networkx` to add the edges to the graph.

In [ ]:
# Your Code goes here!


In [ ]:
# Your Code goes here!


Visualize the graph - this is very ugly!
Use [`draw_networkx`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html) for this.

In [ ]:
# Your Code goes here!


# 3. Calculate graph assortativity

Use the function [`attribute_assortativity_coefficient`](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.assortativity.attribute_assortativity_coefficient.html) of `networkx` to calculate the assortativity with respect to party labels. How high is the value?

In [ ]:
# Your Code goes here!


To see if the assortativity value fits your expectations, use the function [`draw_networkx`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html) to plot the network coloring each node according to the political party label of the politician. Does the pattern of colors fit the value of assortativity?

Hint 1: use the optional function parameters `nodelist` and `node_color` to pass a list of nodes and a list of corresponding colors to the drawing function.  
Hint 2: you can use one of [matplotlibs categorical color maps](https://matplotlib.org/stable/tutorials/colors/colormaps.html) to get a nice series of distinct colors for the parties. 

In [ ]:
# Your Code goes here!


In [ ]:
# Your Code goes here!


# 4. Permutation tests

The above result looks assortative, but how can we test if it could have happened at random and not because of party identity? Here were are going to test it with a permutation test.

First, let's run a permutation. Perform the same assortativity calculation as above but permuting the party labels of nodes. You can do this very efficiently by using `pandas` [`sample`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) function.

Also set the party for each node as node attribute by using [`set_node_attribute`](https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.set_node_attributes.html) (be carefull the parties are now permuted).

In [ ]:
# Your Code goes here!


Is the value much closer to zero?
Repeat the calculation with 1000 permutations and plot the histogram of the resulting values. Add a line with the value of the assortativity without permutation. Is it far or close to the permuted values?

In [ ]:
# Your Code goes here!


To be sure, let's calculate a p-value for the null hypothesis that the assortativity is zero and the alternative hypothesis that it is positive (what we expected):

In [ ]:
# Your Code goes here!


After looking at the above results, do you think it is likely that the assortativity we found in the data was produced by chance?

# 5. Community detection

Let's test if Twitter communities match political affiliations. Remove nodes with degree zero in the network and run the [Louvain community detection algorithm](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.louvain.louvain_communities.html). Visualize the result coloring nodes by community labels.

In [ ]:
# Your Code goes here!


Run the [`modularity`](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.modularity.html) function with the above community labels. Is it high enough to think that the network has a community structure?

In [ ]:
# Your Code goes here!


Repeat but using the party labels instead of the communities detected with Louvain. Is it higher or lower? How far is this modularity from the maximal one found with Louvain?

For this iterate over the parties and filter a subset of users that is in the given party and in the graph. Add the ids of these partymembers (do not include any duplicates) and repeat this for all parties.

Afterwards you can calculate the modularity.

In [ ]:
# Your Code goes here!


Finally, to understand which parties are represented in each community, build a data frame for nodes with two columns: one with the party label and another one with the community label. Use the [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function to print a contingency table. Which party or parties compose each community?

In [ ]:
# Your Code goes here!


# To learn more
* How well can you predict the party of a politician from its neighbors in the network? Here you can use the rule of predicting the party as the majority party among its neighbors and evaluate the accuracy of this approach.
* What would be the results if we use the network of replies? Do you expect assortativity and modularity to be higher or lower?
* If you retrieved data of follower links, you can repeat the above analysis for undirected following relationships. Do you expect a higher or lower assortativity?